# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*. 

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [5]:
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/tp/098qqpfn1r77_knlyqsp_kfm0000gn/T/pip-req-build-0z9hbnn5
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/tp/098qqpfn1r77_knlyqsp_kfm0000gn/T/pip-req-build-0z9hbnn5
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit c2386a7483e6736d53c99c4b56ae5e855fec37ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /private/var/folders/tp/098qqpfn1r77_knlyqsp_kfm0000gn/T/pip-install-ull9yf4f/elegantrl_27bdf2cc66d64783ae979abc1c4141d7
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/ElegantRL.git /private/var/folders/tp/098qqpfn1r77_knlyqsp_kfm0000gn/T/pip-install-ull9yf4f/elegantrl_27bdf2cc66d64783ae979abc1c4141d7
  

In [6]:
import pandas as pd
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS
from finrl.config import *
import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [7]:
TRAIN_START_DATE = '2020-01-01'
TRADE_END_DATE = '2020-01-31'
aapl_df_yf = yf.download(tickers = "aapl", start=TRAIN_START_DATE, end=TRADE_END_DATE)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [8]:
aapl_df_yf.head()

Price           Close       High        Low       Open     Volume
Ticker           AAPL       AAPL       AAPL       AAPL       AAPL
Date                                                             
2020-01-02  72.620834  72.681281  71.373211  71.627084  135480400
2020-01-03  71.914818  72.676447  71.689957  71.847118  146322800
2020-01-06  72.487854  72.526541  70.783256  71.034717  118387200
2020-01-07  72.146942  72.753823  71.926915  72.497529  108872000
2020-01-08  73.307526  73.609760  71.849548  71.849548  132079200

### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [9]:
aapl_df_finrl = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRAIN_END_DATE,
                                ticker_list = ['aapl']).fetch_data()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed

Shape of DataFrame:  (146, 8)


In [10]:
aapl_df_finrl.head()

Price        date      close       high        low       open     volume  \
0      2020-01-02  72.620850  72.681296  71.373226  71.627100  135480400   
1      2020-01-03  71.914795  72.676423  71.689935  71.847095  146322800   
2      2020-01-06  72.487854  72.526541  70.783256  71.034717  118387200   
3      2020-01-07  72.146950  72.753831  71.926922  72.497537  108872000   
4      2020-01-08  73.307526  73.609760  71.849548  71.849548  132079200   

Price   tic  day  
0      aapl    3  
1      aapl    4  
2      aapl    0  
3      aapl    1  
4      aapl    2

## Data for the chosen tickers

In [11]:
config_tickers.DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [12]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2021-10-29'

In [13]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TRADE_END_DATE,
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Shape of DataFrame:  (94301, 8)


In [14]:
df_raw.head()

Price        date      close       high        low       open     volume  \
0      2009-01-02   2.727417   2.736133   2.559414   2.581053  746015200   
1      2009-01-02  40.791466  40.853701  39.934007  40.514866    6547900   
2      2009-01-02  14.929296  15.076040  14.211021  14.342318   10955700   
3      2009-01-02  33.941105  34.173631  32.088407  32.103409    7010200   
4      2009-01-02  30.344681  30.389962  28.921566  29.050940    7117200   

Price   tic  day  
0      AAPL    4  
1      AMGN    4  
2       AXP    4  
3        BA    4  
4       CAT    4

# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [15]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (3228, 8)
Successfully added vix
Successfully added turbulence index


In [16]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [17]:
processed_full.head()

date   tic      close       high        low       open       volume  \
0  2009-01-02  AAPL   2.727417   2.736133   2.559414   2.581053  746015200.0   
1  2009-01-02  AMGN  40.791466  40.853701  39.934007  40.514866    6547900.0   
2  2009-01-02   AXP  14.929296  15.076040  14.211021  14.342318   10955700.0   
3  2009-01-02    BA  33.941105  34.173631  32.088407  32.103409    7010200.0   
4  2009-01-02   CAT  30.344681  30.389962  28.921566  29.050940    7117200.0   

   day  macd   boll_ub   boll_lb  rsi_30     cci_30  dx_30  close_30_sma  \
0  4.0   0.0  2.947758  2.622183   100.0  66.666667  100.0      2.727417   
1  4.0   0.0  2.947758  2.622183   100.0  66.666667  100.0     40.791466   
2  4.0   0.0  2.947758  2.622183   100.0  66.666667  100.0     14.929296   
3  4.0   0.0  2.947758  2.622183   100.0  66.666667  100.0     33.941105   
4  4.0   0.0  2.947758  2.622183   100.0  66.666667  100.0     30.344681   

   close_60_sma        vix  turbulence  
0      2.727417  39.189999         0.0  
1     40.791466  39.189999         0.0  
2     14.929296  39.189999         0.0  
3     33.941105  39.189999         0.0  
4     30.344681  39.189999         0.0

# Part 4: Save the Data

### Split the data for training and trading

In [18]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

83897
9715


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [19]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')